In [1]:
import pandas as pd
import os
import numpy as np
from statistics import mean 
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors

------------------------------ Product Recoommendation -----------------------------------------

In [2]:
df = pd.read_csv('data123_review1000.csv')
df.drop('Unnamed: 0', axis=1, inplace = True)
df.info()#7098

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7098 entries, 0 to 7097
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_name      7098 non-null   object 
 1   product_price     7098 non-null   float64
 2   product_discount  7098 non-null   int64  
 3   average_rating    7098 non-null   float64
 4   fit_rating        7098 non-null   object 
 5   num_review        7098 non-null   int64  
 6   product_tags      7098 non-null   object 
 7   top_3_reviews     7098 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 443.8+ KB


In [3]:
df[df.duplicated(['product_tags','top_3_reviews'])]
df.drop_duplicates(['product_tags','top_3_reviews']).to_csv('data123_review1000_drop_duplicated_tagsreview.csv')

In [4]:
train = pd.read_csv("data123_review1000_drop_duplicated_tagsreview.csv")
train.drop('Unnamed: 0', axis=1, inplace = True)

In [5]:
#train[train.duplicated(['product_name','product_price'])]
train['name_price'] = train['product_name'] + ' $' + train['product_price'].astype(str)
train

,product_name,product_price,product_discount,average_rating,fit_rating,num_review,product_tags,top_3_reviews,name_price
0,SHEIN EZwear Letter Patched Detail Striped Tri...,12.50,-17,4.96,"['Small\n4%', 'True to Size\n96%', 'Large\n0%']",1000,['will repurchase (13)keep warm (16)fast logis...,['Why are you still looking!!??? This is your ...,SHEIN EZwear Letter Patched Detail Striped Tri...
1,SHEIN EZwear Contrast Trim Lantern Sleeve Dust...,17.49,0,4.94,"['Small\n3%', 'True to Size\n97%', 'Large\n0%']",1000,['will repurchase (2)keep warm (12)fall outfit...,['PLEASE LIKE IF YOU FIND THIS HELPFUL I NEED ...,SHEIN EZwear Contrast Trim Lantern Sleeve Dust...
2,SHEIN ICON Rhinestone Butterfly Pattern Drawst...,15.99,0,4.92,"['Small\n3%', 'True to Size\n95%', 'Large\n2%']",1000,['will repurchase (2)no color difference (4)ke...,['so cute and comfy. the pockets are so big an...,SHEIN ICON Rhinestone Butterfly Pattern Drawst...
3,SHEIN LUNE Plaid Print Drop Shoulder Shirt,9.49,0,4.88,"['Small\n1%', 'True to Size\n90%', 'Large\n9%']",1000,['fall outfits (500+)will repurchase (16)fit w...,['(((LIKE THIS IF YOU READ IT SO YOUR GIRL CAN...,SHEIN LUNE Plaid Print Drop Shoulder Shirt $9.49
4,DAZY Zip Up Drop Shoulder Drawstring Thermal L...,23.99,-13,4.93,"['Small\n1%', 'True to Size\n90%', 'Large\n9%']",1000,['keep warm (21)will repurchase (1)fast logist...,['I LOVE THIS SM ITS SO COMFY AND BIG OMG. i l...,DAZY Zip Up Drop Shoulder Drawstring Thermal L...
...,...,...,...,...,...,...,...,...,...
5165,SHEIN EZwear High Waist Ripped Boyfriend Jeans,25.99,0,4.77,"['Small\n5%', 'True to Size\n86%', 'Large\n9%']",1000,['will repurchase (9)love (1000+)fit well (100...,['I LOVE THEM. GET THEM I GOT THIS WITH NO REV...,SHEIN EZwear High Waist Ripped Boyfriend Jeans...
5166,SHEIN MOD Ruched Bandana Hem Mesh Tube Top,6.99,0,4.87,"['Small\n5%', 'True to Size\n93%', 'Large\n2%']",1000,['will repurchase (31)full cup size (47)no col...,['Wore this at school on my birthday super cut...,SHEIN MOD Ruched Bandana Hem Mesh Tube Top $6.99
5167,DAZY Solid Drop Shoulder Ribbed Knit Tee,9.99,0,4.86,"['Small\n2%', 'True to Size\n93%', 'Large\n5%']",1000,['will repurchase (4)keep warm (4)Comfortable ...,['It is very see through but i like to use it ...,DAZY Solid Drop Shoulder Ribbed Knit Tee $9.99
5168,SHEIN MOD Solid Ruched Bustier Cami Top,5.49,0,4.91,"['Small\n5%', 'True to Size\n95%', 'Large\n0%']",1000,['will repurchase (12)good fabric material (10...,"['', 'Simple black date night top. Goes well w...",SHEIN MOD Solid Ruched Bustier Cami Top $5.49


In [6]:
train[train.duplicated(['name_price'])]#1222
train.drop_duplicates(['name_price'],inplace=True)

In [7]:
def process_tag(product_tags):
    tag = []
    tmp = list(filter(lambda x: len(x)>0 and x != '\'' and x != '\"',
                          re.split(r'[(+\[\])]',product_tags)))
    s = ''
    for j in range(0, len(tmp), 2):
        tmp[j] = tmp[j].replace(' ','')
        if tmp[j].find('\'') == 0:
            tmp[j] = tmp[j].replace('\'','')
        if tmp[j].find('\"') == 0:
            tmp[j] = tmp[j].replace('\"','')
        for k in range(int(tmp[j+1])):
            s += tmp[j] + ' '
        print(s)
        print(tag)
    tag.append(s)
    return tag

In [8]:
train['str_product_tags'] = train['product_tags'].apply(process_tag)
#train

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
data = train[['name_price','product_discount','average_rating','str_product_tags']]

In [10]:
data['str_product_tags'] = [''.join(map(str, l)) for l in data['str_product_tags']]

/var/folders/6j/n3f0x1ws4s9fc5xsw45tr2sh0000gn/T/ipykernel_44436/1516071446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['str_product_tags'] = [''.join(map(str, l)) for l in data['str_product_tags']]


------------------------- cosine similarity -------------------------

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(data['str_product_tags'])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)
print(similarities)

[[1.         0.18974973 0.17651442 ... 0.12792887 0.81656422 0.25796683]
 [0.18974973 1.         0.53167325 ... 0.33832613 0.34674528 0.33810008]
 [0.17651442 0.53167325 1.         ... 0.71749258 0.48061347 0.63381849]
 ...
 [0.12792887 0.33832613 0.71749258 ... 1.         0.4625586  0.44985657]
 [0.81656422 0.34674528 0.48061347 ... 0.4625586  1.         0.58516892]
 [0.25796683 0.33810008 0.63381849 ... 0.44985657 0.58516892 1.        ]]


In [13]:
op = pd.DataFrame(similarities, index = data['name_price'],columns = data['name_price'])

In [14]:
#input #review !=1000: SHEIN LUNE Batwing Sleeve Pocket Patched Shirt
#input #review =1000: SHEIN Frenchy Color Block Drop Shoulder Sweater

input_product_name = 'SHEIN EZwear Letter Patched Detail Striped Trim Drop Shoulder Crop Varsity Jacket $12.5'
recommendations = op.nlargest(10,input_product_name)[input_product_name].to_frame(name = 'similarity rate').reset_index()
recommendations

,name_price,similarity rate
0,SHEIN EZwear Letter Patched Detail Striped Tri...,1.000000
1,SHEIN EZwear Letter Patched Detail Striped Tri...,0.999991
2,SHEIN EZwear Bleach Wash Flare Leg Jeans $20.82,0.996892
3,SHEIN Privé Sheer Asymmetrical Hem Cami Top $5.99,0.980994
4,SHEIN Coolane Letter & Building Print Notched ...,0.979352
5,SHEIN EZwear Solid Ribbed Knit Pencil Skirt $6.99,0.979025
6,SHEIN PETITE Backless Tube Unitard Romper $4.58,0.978267
7,Ribbed Bikini Set Contrast Binding Ring Linked...,0.978180
8,SHEIN EZwear Drawstring Hem Vest Puffer Coat $...,0.977426
9,Ribbed Bikini Set Contrast Binding Ring Linked...,0.977178


I realized that I did not handle the duplicates very well. Considering the situation that there are duplicated pairs of product name and top 3 reviews for same product in different colors. Cosine similarity will not work for this datset, because cosine similarity relies on the column name, which is the input of product name, to find the top n-th products that align with the input. Since with have duplicated product name, it will lead to error when matching the columns. For now, I concate the product name and the price, and dropped all the duplicated pairs of name and price. 

Thus, We would better try KNN model instead.

Before that, I would like to try apply cosine similarity to concated tags, name and reviews. 

Improvenment:
In order to avoid this problem raise in the future project, I should thought through what would be the unique identifier for each item before scrapping data.

In [15]:
train['all_text'] = train[['name_price','str_product_tags','top_3_reviews']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(train['all_text'].head())

0    SHEIN EZwear Letter Patched Detail Striped Tri...
1    SHEIN EZwear Contrast Trim Lantern Sleeve Dust...
2    SHEIN ICON Rhinestone Butterfly Pattern Drawst...
3    SHEIN LUNE Plaid Print Drop Shoulder Shirt $9....
4    DAZY Zip Up Drop Shoulder Drawstring Thermal L...
Name: all_text, dtype: object


In [16]:
vectorizer1 = CountVectorizer()
vectorized1 = vectorizer1.fit_transform(train['all_text'])
similarities1 = cosine_similarity(vectorized1)
print(similarities1)

[[1.         0.19127133 0.17757475 ... 0.13078629 0.81630848 0.25896944]
 [0.19127133 1.         0.53518673 ... 0.35313786 0.34980789 0.34007961]
 [0.17757475 0.53518673 1.         ... 0.72342451 0.48048891 0.63188394]
 ...
 [0.13078629 0.35313786 0.72342451 ... 1.         0.46784715 0.45372571]
 [0.81630848 0.34980789 0.48048891 ... 0.46784715 1.         0.58546627]
 [0.25896944 0.34007961 0.63188394 ... 0.45372571 0.58546627 1.        ]]


In [17]:
op = pd.DataFrame(similarities1, index = train['name_price'],columns = train['name_price'])
input_product_name = 'SHEIN EZwear Letter Patched Detail Striped Trim Drop Shoulder Crop Varsity Jacket $12.5'
recommendations2 = op.nlargest(10,input_product_name)[input_product_name].to_frame(name = 'similarity rate').reset_index()
recommendations2

,name_price,similarity rate
0,SHEIN EZwear Letter Patched Detail Striped Tri...,1.000000
1,SHEIN EZwear Letter Patched Detail Striped Tri...,0.999986
2,SHEIN EZwear Bleach Wash Flare Leg Jeans $20.82,0.996494
3,SHEIN Privé Sheer Asymmetrical Hem Cami Top $5.99,0.980613
4,SHEIN Coolane Letter & Building Print Notched ...,0.978956
5,SHEIN EZwear Solid Ribbed Knit Pencil Skirt $6.99,0.978717
6,SHEIN PETITE Backless Tube Unitard Romper $4.58,0.977846
7,Ribbed Bikini Set Contrast Binding Ring Linked...,0.977831
8,SHEIN EZwear Drawstring Hem Vest Puffer Coat $...,0.977132
9,Ribbed Bikini Set Contrast Binding Ring Linked...,0.976833


In [18]:
recommendations.compare(recommendations2,keep_shape=True,keep_equal=True,result_names=("1","2"))

name_price  \
                                                   1   
0  SHEIN EZwear Letter Patched Detail Striped Tri...   
1  SHEIN EZwear Letter Patched Detail Striped Tri...   
2    SHEIN EZwear Bleach Wash Flare Leg Jeans $20.82   
3  SHEIN Privé Sheer Asymmetrical Hem Cami Top $5.99   
4  SHEIN Coolane Letter & Building Print Notched ...   
5  SHEIN EZwear Solid Ribbed Knit Pencil Skirt $6.99   
6    SHEIN PETITE Backless Tube Unitard Romper $4.58   
7  Ribbed Bikini Set Contrast Binding Ring Linked...   
8  SHEIN EZwear Drawstring Hem Vest Puffer Coat $...   
9  Ribbed Bikini Set Contrast Binding Ring Linked...   

                                                     similarity rate            
                                                   2               1         2  
0  SHEIN EZwear Letter Patched Detail Striped Tri...        1.000000  1.000000  
1  SHEIN EZwear Letter Patched Detail Striped Tri...        0.999991  0.999986  
2    SHEIN EZwear Bleach Wash Flare Leg Jeans $20.82        0.996892  0.996494  
3  SHEIN Privé Sheer Asymmetrical Hem Cami Top $5.99        0.980994  0.980613  
4  SHEIN Coolane Letter & Building Print Notched ...        0.979352  0.978956  
5  SHEIN EZwear Solid Ribbed Knit Pencil Skirt $6.99        0.979025  0.978717  
6    SHEIN PETITE Backless Tube Unitard Romper $4.58        0.978267  0.977846  
7  Ribbed Bikini Set Contrast Binding Ring Linked...        0.978180  0.977831  
8  SHEIN EZwear Drawstring Hem Vest Puffer Coat $...        0.977426  0.977132  
9  Ribbed Bikini Set Contrast Binding Ring Linked...        0.977178  0.976833

As we observed, cancating product name with tags, review does not make much difference on the output product, but only caused slightly changed in the similarity rate.

In the next noteboook, I will build KNN model, adjust parameters and evaluate its performance and accuracy.  